In [9]:
import requests
from bs4 import BeautifulSoup

# Function to check HTTPS
def check_https(url):
    if url.startswith('https'):
        print("HTTPS is enabled.")
    else:
        print("HTTPS is not enabled.")

# Function to find login or signup links
def find_login_signup_links(base_url):
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    login_links = []
    signup_links = []

    for link in soup.find_all('a', href=True):
        href = link['href']
        if 'login' in href.lower():
            login_links.append(href)
        if 'signup' in href.lower() or 'register' in href.lower():
            signup_links.append(href)

    return login_links, signup_links

# Function to analyze login forms
def analyze_login_form(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find input fields
    inputs = soup.find_all('input')
    for input_field in inputs:
        print(f"Input name: {input_field.get('name')}, type: {input_field.get('type')}")

    # Check for CAPTCHA
    captcha = soup.find('div', {'class': 'g-recaptcha'})
    if captcha:
        print("CAPTCHA is present.")
    else:
        print("CAPTCHA is not present.")

# Function to inspect cookies and tokens
def inspect_cookies_and_tokens(session, login_url, username, password):
    login_payload = {
        'username': username,
        'password': password
    }
    login_response = session.post(login_url, data=login_payload)

    # Check cookies
    for cookie in session.cookies:
        print(f"Cookie: {cookie.name}, Secure: {cookie.secure}, HttpOnly: {cookie.has_nonstandard_attr('HttpOnly')}")

    # Check for JWT tokens in headers
    auth_header = login_response.headers.get('Authorization')
    if auth_header and 'Bearer' in auth_header:
        print("JWT token is used.")
    else:
        print("JWT token is not used.")

def analyze_session_management(session):
    # Check session ID length and randomness
    session_id = session.cookies.get('sessionid')
    if session_id and len(session_id) > 20:
        print("Session ID is long and likely random.")
    else:
        print("Session ID is short or not random.")

    # Check session expiration
    session_cookie = session.cookies.get('sessionid', domain='bekushal.com')
    if session_cookie:
        session_expiry = session_cookie.expires
        if session_expiry:
            print(f"Session expires at: {session_expiry}")
        else:
            print("Session expiration is not set.")
    else:
        print("Session cookie not found.")


# Function to perform authorization checks
def authorization_checks(session, protected_url):
    protected_response = session.get(protected_url)

    if protected_response.status_code == 403:
        print("Access is restricted based on user roles.")
    elif protected_response.status_code == 200:
        print("Access is granted.")
    else:
        print("Unexpected response status code.")

# Function to review HTTP headers
def review_http_headers(response):
    headers = response.headers

    security_headers = [
        'Strict-Transport-Security',
        'Content-Security-Policy',
        'X-Content-Type-Options',
        'X-Frame-Options'
    ]

    for header in security_headers:
        if header in headers:
            print(f"{header} is present.")
        else:
            print(f"{header} is not present.")

# Main function to run all checks
def main():
    base_url = 'https://amazon.in'
    session = requests.Session()

    # Check HTTPS
    check_https(base_url)

    # Find login or signup links
    login_links, signup_links = find_login_signup_links(base_url)
    print(f"Login links found: {login_links}")
    print(f"Signup links found: {signup_links}")

    # Analyze login form if login link is found
    if login_links:
        analyze_login_form(base_url + login_links[0])

    # Inspect cookies and tokens if login link is found
    if login_links:
        username = 'your_username'
        password = 'your_password'
        inspect_cookies_and_tokens(session, base_url + login_links[0], username, password)

    # Analyze session management
    analyze_session_management(session)

    # Perform authorization checks
    protected_url = base_url + '/protected'  # Adjust this as needed
    authorization_checks(session, protected_url)

    # Review HTTP headers
    response = session.get(base_url)
    review_http_headers(response)

if __name__ == "__main__":
    main()


HTTPS is enabled.
Login links found: []
Signup links found: []
Session ID is short or not random.
Session cookie not found.
Unexpected response status code.
Strict-Transport-Security is present.
Content-Security-Policy is present.
X-Content-Type-Options is present.
X-Frame-Options is present.


In [12]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://sitare.org'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Extract emails
emails = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', soup.text)
print('Emails:', emails)

# Extract phone numbers
phone_numbers = re.findall(r'\b\d{10}\b', soup.text)
print('Phone Numbers:', phone_numbers)

# Extract privacy policy URL
privacy_policy = soup.find('a', href=True, text=re.compile(r'Privacy Policy', re.I))
if privacy_policy:
    privacy_policy_url = privacy_policy['href']
    print(f'Privacy Policy URL: {privacy_policy_url}')


Emails: []
Phone Numbers: []


<ipython-input-12-c5d4151b2cff>:18: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  privacy_policy = soup.find('a', href=True, text=re.compile(r'Privacy Policy', re.I))
